In [41]:
from DBcm import UseDatabase
import os
from datetime import datetime, timedelta

dataset_path = r""


def ingest(dataset, debug):
    config = {
        "host": "localhost",
        "database": "swimmers_webapp",
        "user": "admin",
        "password": "swimmerspwd"
    }

    files = get_files(dataset)
    if ".DS_Store" in files:
        files.remove(".DS_Store")


    # start db connection
    with UseDatabase(config) as cursor:
        swimmer_data = set()
        events = set()
        
        for f in files:
            f = f.removesuffix(".txt")
            name, age, distance, stroke = f.split("-")
            distance = distance.removesuffix("m")
            distance = int(distance)
            inserted = True
            
            if (distance, stroke) not in events:
                events.add((distance, stroke))
                _SQL = f"""insert ignore into events (distance, stroke) values ({distance}, '{stroke}');"""
                cursor.execute(_SQL)
            
            if (name, age) not in swimmer_data:
                swimmer_data.add((name, age))
                _SQL = f"""insert ignore into swimmers (name, age) values ('{name}', {age});"""
                cursor.execute(_SQL)
            
            _SQL = f"""select id from swimmers where name = '{name}' and age = {int(age)};"""
            cursor.execute(_SQL)
            swimmer_id = cursor.fetchall()[0][0]
            
            _SQL = f"""select id from events where distance = {distance} and stroke = '{stroke}';"""
            cursor.execute(_SQL)
            event_id = cursor.fetchall()[0][0]
    
            lines = read_file(dataset, f"{f}.txt")
            for line in lines:
                times = line.split(",")
                for time in times:
                    time = time.strip()
                    _SQL = f"""insert ignore into times (swimmer_id, event_id, time) VALUES ({swimmer_id}, {event_id}, '{time}');"""
                    cursor.execute(_SQL)
                
            
def get_files(dataset):
    path = rf"{dataset_path}{dataset}"
    return os.listdir(path)


def read_file(dataset, filename):
    path = rf"{dataset_path}{dataset}/{filename}"
    with open(path) as f:
        return f.readlines()


def get_raw_information(swimmer: str, age: int, distance: int, stroke: str):
    config = {
        "host": "localhost",
        "database": "swimmers_webapp",
        "user": "admin",
        "password": "swimmerspwd"
    }

    with UseDatabase(config) as cursor:
        _SQL = f"""select id from swimmers where name = '{swimmer}' and age = {age} limit 1;"""
        cursor.execute(_SQL)
        swimmer_id = cursor.fetchone()[0]
        
        _SQL = f"""select event_id from times where swimmer_id = {swimmer_id}"""
        cursor.execute(_SQL)
        results = cursor.fetchall()
        event_ids = set()
        # convert arr of tuples into array of first entry in tuple
        for result in results:
            event_ids.add(result[0])
            
        return swimmer_id, event_ids


def get_human_readable_information(raw_data: tuple):
    swimmer_id = raw_data[0]
    event_ids = raw_data[1]
    
    config = {
        "host": "localhost",
        "database": "swimmers_webapp",
        "user": "admin",
        "password": "swimmerspwd"
    }
    
    dataset = list()
    
    with UseDatabase(config) as cursor:
        for event, i in enumerate(event_ids):
            _SQL = f"""select * from times where swimmer_id = {swimmer_id} and event_id = {event} limit 1"""
            cursor.execute(_SQL)
            data = cursor.fetchall()
            # data_string = f"""
            # Swimmer ID: {data[i][0]}
            # Event ID: {data[i][1]}
            # Time: {data[i][2]}
            # Timestamp: {data[i][3]}
            # """
            
    return data
            

ingest("dataset_1", False)
ingest ("dataset_2", False)

darius = get_raw_information("Darius", 8, 50, "Breast")
darius_dataset = get_human_readable_information(darius)
darius_dataset

[(25, 2, '36.13', datetime.datetime(2023, 11, 30, 16, 33, 38))]